In [3]:
%pip install pygame

   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---- ----------------------------------- 1.3/10.6 MB 7.4 MB/s eta 0:00:02
   -------- ------------------------------- 2.4/10.6 MB 6.1 MB/s eta 0:00:02
   ---------- ----------------------------- 2.9/10.6 MB 5.1 MB/s eta 0:00:02
   ------------ --------------------------- 3.4/10.6 MB 4.4 MB/s eta 0:00:02
   ------------- -------------------------- 3.7/10.6 MB 4.1 MB/s eta 0:00:02
   -------------- ------------------------- 3.9/10.6 MB 3.5 MB/s eta 0:00:02
   --------------- ------------------------ 4.2/10.6 MB 2.9 MB/s eta 0:00:03
   ---------------- ----------------------- 4.5/10.6 MB 2.7 MB/s eta 0:00:03
   ----------------- ---------------------- 4.7/10.6 MB 2.5 MB/s eta 0:00:03
   ------------------ --------------------- 5.0/10.6 MB 2.4 MB/s eta 0:00:03
   ------------------- -------------------- 5.2/10.6 MB 2.2 MB/s eta 0:00:03
   ------------------- -------------------- 5.2/10.6 MB 2.2 MB/s eta 0:00:03
   ---

In [14]:
import pygame
import numpy as np
import torch
import random

# Constants
screen_width = 800
screen_height = 600
ball_size = 10
paddle_width = 10
paddle_height = 100
white = (255, 255, 255)
black = (0, 0, 0)

# Setup CUDA device if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class Ball:
    def __init__(self, x, y, size, speed=7):
        self.x = x
        self.y = y
        self.size = size
        self.speed = speed
        self.dx = speed
        self.dy = speed
        self.rect = pygame.Rect(x, y, size, size)

    def move(self):
        self.x += self.dx
        self.y += self.dy
        self.rect.x = self.x
        self.rect.y = self.y

    def bounce(self):
        if self.y <= 0 or self.y >= screen_height - self.size:
            self.dy *= -1
        
class Paddle:
    def __init__(self, x, y, width, height, speed=5):
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        self.speed = speed
        self.rect = pygame.Rect(x, y, width, height)
        self.score = 0

    def move(self, up=True):
        if up and self.y > 0:
            self.y -= self.speed
        elif not up and self.y < screen_height - self.height:
            self.y += self.speed
        self.rect.y = self.y

    def ai_move(self, ball):
        if self.y + self.height/2 < ball.y:
            self.move(False)
        elif self.y + self.height/2 > ball.y:
            self.move(True)

# Initialize Pygame and font
pygame.init()
pygame.font.init()

# Create display and clock before game objects
screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption("Pong with RL")
clock = pygame.time.Clock()

# Create game objects
ball = Ball(screen_width//2, screen_height//2, ball_size)
player_paddle = Paddle(50, screen_height//2 - paddle_height//2, paddle_width, paddle_height)
opponent_paddle = Paddle(screen_width - 50 - paddle_width, screen_height//2 - paddle_height//2, 
                        paddle_width, paddle_height)

# Initialize game state
running = True
# Game states for RL
state_size = 6  # ball_x, ball_y, ball_dx, ball_dy, paddle_y, opponent_y
action_size = 3  # up, down, stay

# Main game loop
while running:
    # Handle events
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Handle paddle movement
    keys = pygame.key.get_pressed()
    if keys[pygame.K_UP]:
        player_paddle.move(True)
    if keys[pygame.K_DOWN]:
        player_paddle.move(False)

    # Move ball and AI opponent
    ball.move()
    opponent_paddle.ai_move(ball)

    # Check for collisions
    if ball.rect.colliderect(player_paddle.rect) or ball.rect.colliderect(opponent_paddle.rect):
        ball.dx *= -1

    # Ball bouncing off top and bottom
    ball.bounce()

    # Score points
    if ball.x <= 0:
        opponent_paddle.score += 1
        ball.x = screen_width//2
        ball.y = screen_height//2
        ball.dx = ball.speed
        ball.dy = ball.speed
    elif ball.x >= screen_width:
        player_paddle.score += 1
        ball.x = screen_width//2
        ball.y = screen_height//2
        ball.dx = -ball.speed
        ball.dy = ball.speed
    # Draw everything
    screen.fill(black)
    pygame.draw.rect(screen, white, player_paddle.rect)
    pygame.draw.rect(screen, white, opponent_paddle.rect)
    pygame.draw.rect(screen, white, ball.rect)
    pygame.draw.rect(screen, white, opponent_paddle.rect)
    pygame.draw.rect(screen, white, ball.rect)

    # Display scores
    font = pygame.font.Font(None, 74)
    score_text = font.render(f"{player_paddle.score} - {opponent_paddle.score}", True, white)
    screen.blit(score_text, (screen_width//2 - score_text.get_width()//2, 20))

    # Update display
    pygame.display.flip()

    # Control game speed
    ball.x = random.randint(0, screen_width // 4)
    ball.y = random.randint(0, screen_height - ball_size)
    ball.rect.x = ball.x
    ball.rect.y = ball.y
    clock.tick(30)  # Slowing down the game to visualize better
    clock.tick(60)

# Quit pygame when the game loop ends
pygame.quit()